In [12]:

apis_list = ast.literal_eval(APIS_LIST)

In [ ]:
discourse_imgs = {}
files = [*Path("raw-data/Discourse-data").glob("*.json")]
for file_path in files:
    data = read_json_file(file_path)
    posts = data.get('post_stream', {}).get('posts', [])
    topic_id = data.get('id')
    topic_slug = data.get('slug', '')
    
    for post in posts:
        content = post.get('cooked', '')
        question_img_url = extract_europe1_urls(content) if extract_europe1_urls(content) else None
        
        if question_img_url:
            url = question_img_url[0]
            print(f'\n processing the image {url} \n ')
            b64IMG = image_url_to_base64(url)
            # b64IMG = True
            if b64IMG:
                try:
                    description = await describe_base64_image(b64IMG, apis_list[count_api_calls % 9],3)
                    print(description)
                    count_api_calls += 1
                    # description = 'flajfljfl;aj;faj;l'
                    discourse_imgs[url] = description
                except Exception as e:
                    print(f"Error describing image {url}: {e}")
                finally:
                    with open('img_description.txt', 'w') as f:
                        f.write(str(discourse_imgs))
            else:
                print(f"Skipping image {url} due to download error.")



 processing the image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png 
 
Error describing image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png: name 'count_api_calls' is not defined

 processing the image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/7/b/7be5e1eb5ea66bf30d3a3ebb32850f99e077bcb5.png 
 
Error describing image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/7/b/7be5e1eb5ea66bf30d3a3ebb32850f99e077bcb5.png: name 'count_api_calls' is not defined

 processing the image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png 
 
Error describing image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png: name 'count_api_calls' is not defined

 processing the image https://europe1.d

In [ ]:
discourse_imgs

In [9]:
import base64
import requests
from io import BytesIO
from PIL import Image
from google.genai import Client, types

In [8]:
def image_url_to_base64(image_url, format_hint=None):
    response = requests.get(image_url)
    response.raise_for_status()  # Ensure the request was successful
    img = Image.open(BytesIO(response.content))
    buffered = BytesIO()
    # Use the format_hint if provided, else use the format from the image
    img_format = format_hint if format_hint else img.format
    img.save(buffered, format=img_format)
    img_bytes = buffered.getvalue()
    base64_str = base64.b64encode(img_bytes).decode('utf-8')
    return base64_str

# Example usage:
url = "https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png"
b64_img = image_url_to_base64(url)
print(b64_img[:100])  # Print the first 100 characters of the base64 string

iVBORw0KGgoAAAANSUhEUgAABRAAAACeCAMAAAB9wNa5AAACcFBMVEUhJSne4ubSsn6n1uYhJX5kJSne1rVkstqJxObexJ2Jmsi6


In [13]:
def describe_base64_image(base64_image, api_key):
    # Decode the base64 string to bytes
    image_bytes = base64.b64decode(base64_image)
    
    # Initialize the Gemini API client
    client = Client(api_key=api_key)
    
    # Generate the description
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=[
            types.Part.from_bytes(
                data=image_bytes,
                mime_type='image/jpeg',  # or 'image/png' if your image is PNG
            ),
            """Perform Optical Character Recognition (OCR) on the provided image to extract all readable text accurately. Follow these steps:

            1. Analyze the input image or document containing text
            2. Extract ALL visible text while preserving formatting, line breaks, and spatial relationships where possible
            3. Identify and transcribe any error messages, menu options, button labels, dialog boxes, and interface elements
            4. Verify the accuracy of recognized characters, especially for technical terms, error codes, and numerical values
            5. Present the extracted text in a clear, structured format that maintains the original context

            ## Output Requirements:
            - Return extracted text as plain, continuous text with natural spacing
            - Preserve original line breaks and formatting structure
            - Handle various fonts, sizes, and orientations of text
            - Be robust against noise, distortion, and low image quality
            - Flag any unclear or illegible parts with [UNCLEAR] markers
            - Do not include additional commentary or metadata
            """
        ]
    )
    return response

description = describe_base64_image(b64_img, apis_list[8])

In [15]:
description.text

'16 Move and rename files (0.5 marks)\nDownload q-move-rename-files.zip and extract it. Use mv to move all files under folders into an empty folder. Then rename all files replacing each digit with the next (i.e. a19b.txt\nbecomes a20b.txt)\nWhat does running grep .* | LC_ALL=C sort | sha256sum in bash on that folder show?\n'

In [17]:
from config import APIS_LIST
from helper import extract_europe1_urls, read_json_file
from pathlib import Path

In [21]:
discourse_imgs = {}
files = [*Path("raw-data/Discourse-data").glob("*.json")]
count_api_calls = 0

for file_path in files:
    try:
        data = read_json_file(file_path)
        posts = data.get('post_stream', {}).get('posts', [])
        topic_id = data.get('id')
        topic_slug = data.get('slug', '')
    except Exception as e:
        print(f"Error reading/parsing file {file_path}: {e}")
        continue

    for post in posts:
        try:
            content = post.get('cooked', '')
            question_img_url = extract_europe1_urls(content) if extract_europe1_urls(content) else None

            if question_img_url:
                url = question_img_url[0]
                print(f'\n processing the image {url} \n ')
                b64IMG = image_url_to_base64(url)
                description = describe_base64_image(b64IMG, apis_list[count_api_calls % 9])
                print(description.text)
                count_api_calls += 1
                discourse_imgs[url] = description.text

                with open('img_description.txt', 'w', encoding="utf-8") as f:
                    f.write(str(discourse_imgs))
        except Exception as e:
            print(f"Error processing post/image in file {file_path}: {e}")
            continue



 processing the image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png 
 
16 Move and rename files (0.5 marks)
Download q-move-rename-files.zip and extract it. Use mv to move all files under folders into an empty folder. Then rename all files replacing each digit with the next (i.e. a19b.txt
becomes a20b.txt)
What does running grep .* | LC_ALL=C sort | sha256sum in bash on that folder show?


 processing the image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/7/b/7be5e1eb5ea66bf30d3a3ebb32850f99e077bcb5.png 
 
Tools in Data Science - Graded Assignment 1
Deadline:
Note: Every page reload randomizes the quiz. So don't copy answers from previous attempts. You can submit multiple times.


 processing the image https://europe1.discourse-cdn.com/flex013/uploads/iitm/original/3X/f/8/f8af7c6e2fe001c8bf000605b52c903e1b0e6fe1.png 
 
16 Move and rename files (0.5 marks)
Download q-move-rename-files.zip and ex